In [1]:
# data prepare
import glob,re,os,sys
os.chdir('C:/kongjy_special/实习工作内容总结/201805-201808华泰证券/Task4/')
sys.path.append('C:\\kongjy_special\\实习工作内容总结\\201805-201808华泰证券\\Task4')
from matplotlib import pyplot as plt
import pandas as pd
import time
import TranFeature,Tools_kjy
import warnings

warnings.filterwarnings('ignore')
pd.options.display.width = 200

# 文件名
TickNameList = glob.glob('Java_log_orders_600519/Tick*.csv')
TranNameList = glob.glob('Java_log_orders_600519/Tran*.csv')
# TestName是 开平仓点 的csv，需要和Tick以及Tran保持时间对齐
DateList = [re.search('\d{8}',i)[0] for i in TickNameList if re.search('\d{8}',i)]  
TestNameFun = lambda x: "Java_log_orders_600519\\600519.SH_back_test_info_" + x +".csv"
TestNameList = [TestNameFun(i) for i in DateList]
print (TickNameList[0])
print (TranNameList[0])
print (TestNameList[0])

Java_log_orders_600519\Tick20180502.csv
Java_log_orders_600519\Tran20180502.csv
Java_log_orders_600519\600519.SH_back_test_info_20180502.csv


In [110]:
# 数据读入 和 预处理
date_index = 8
date = re.search('\d{8}',TickNameList[date_index])[0]
tempTick = pd.read_csv(TickNameList[date_index])
tempTran = pd.read_csv(TranNameList[date_index])
tempTest = pd.read_csv(TestNameList[date_index])
print (TickNameList[date_index])
print (TranNameList[date_index])
print (TestNameList[date_index])
# Tick数据处理
tempTick = tempTick.rename(columns={"Unnamed: 0":"Timestamp"})
tempTick['Timestamp'] = pd.to_datetime(tempTick['Timestamp'])
tempTick['median_5'] = (tempTick['Price'].rolling(window=5).median()).shift(-2).fillna(method='pad')
tempTick['median_5'],_ = Tools_kjy.smooth(tempTick['median_5'])
# 开平仓点提取
tickindex = tempTest[tempTest.order != " "].index
print ("Date : %s"%(date))
# # Tran 数据处理
tempTran = tempTran.rename(columns={"Unnamed: 0":"Timestamp"})
tempTran['Timestamp'] = pd.to_datetime(tempTran['Timestamp'])
FeatureTran = TranFeature.TranFeature(tempTran,tempTick)
tempTick = FeatureTran._Tran_features()

# past change
def past_change(x):
    tempTick['Past_diff'+str(x)] = tempTick['Price'].diff(x).fillna(0)
past_change(1);past_change(2);past_change(3);past_change(4);past_change(5);past_change(6)

# future change
tempTick['diff_3Tick'] = tempTick['Price'].diff(3).shift(-3).fillna(0)
tempTick['diff_1Tick'] = tempTick['Price'].diff(1).shift(-1).fillna(0)
tempTick['isadd_3Tick'] = tempTick['diff_3Tick'].apply(Tools_kjy.trinary)
tempTick['isadd_1Tick'] = tempTick['diff_1Tick'].apply(Tools_kjy.trinary)

print (tempTick.iloc[:20,-10:])


Java_log_orders_600519\Tick20180514.csv
Java_log_orders_600519\Tran20180514.csv
Java_log_orders_600519\600519.SH_back_test_info_20180514.csv
Date : 20180514
_drop_Contunous costs 0.087 seconds!
    Past_diff1  Past_diff2  Past_diff3  Past_diff4  Past_diff5  Past_diff6  diff_3Tick  diff_1Tick  isadd_3Tick  isadd_1Tick
0         0.00        0.00        0.00        0.00        0.00        0.00       -0.98       -0.98            0            0
1        -0.98        0.00        0.00        0.00        0.00        0.00        0.62        0.94            2            2
2         0.94       -0.04        0.00        0.00        0.00        0.00       -0.94       -0.94            0            0
3        -0.94        0.00       -0.98        0.00        0.00        0.00        0.98        0.62            2            2
4         0.62       -0.32        0.62       -0.36        0.00        0.00        0.38       -0.62            2            0
5        -0.62        0.00       -0.94        0.00      

In [111]:
# 下单点的各个情况
tempTick.iloc[tickindex,-10:]

,Past_diff1,Past_diff2,Past_diff3,Past_diff4,Past_diff5,Past_diff6,diff_3Tick,diff_1Tick,isadd_3Tick,isadd_1Tick
20,-0.20,0.00,0.72,0.50,0.50,-0.37,0.03,-0.50,2,0
21,-0.50,-0.70,-0.50,0.22,0.00,0.00,0.50,0.70,2,2
22,0.70,0.20,0.00,0.20,0.92,0.70,-0.70,-0.17,0,0
25,-0.50,-0.53,-0.70,0.00,-0.50,-0.70,0.18,0.15,2,2
32,0.08,0.18,0.20,0.50,0.53,0.53,0.19,-0.18,2,0
33,-0.18,-0.10,0.00,0.02,0.32,0.35,0.40,0.00,2,1
34,0.00,-0.18,-0.10,0.00,0.02,0.32,0.50,0.37,2,2
35,0.37,0.37,0.19,0.27,0.37,0.39,0.43,0.03,2,2
36,0.03,0.40,0.40,0.22,0.30,0.40,0.60,0.10,2,2
37,0.10,0.13,0.50,0.50,0.32,0.40,0.99,0.30,2,2


In [113]:
# 下面提取的特征是针对时间序列的
# 盘口等量
Tools_kjy.cumsum(tempTick)
Tools_kjy.findequal(tempTick,5)
Tools_kjy.findequal(tempTick,10)
Tools_kjy.findequal(tempTick,15)
Tools_kjy.findequal(tempTick,20)
print ("done")

done


In [100]:
# 统计买卖盘的特征
import numpy as np
tempTick['CountBid'] = tempTick['CountBid'].clip(0,12).apply(int)
tempTick['CountAsk'] = tempTick['CountAsk'].clip(0,12).apply(int)
groups = tempTick[['CountBid','CountAsk','diff_3Tick','diff_1Tick']].groupby(['CountBid','CountAsk'])
groupsCountBA = groups.agg({'diff_1Tick':[np.mean,np.std,np.max,np.min,np.median,np.size]}).reset_index()
groupsCountBA.columns = ['CountBid','CountAsk',' mean','std','amax','amin','median','size']
print (groupsCountBA.head(20))

    CountBid  CountAsk      mean       std  amax  amin  median   size
0          0         0  0.001316  0.198773  1.79 -1.50   0.000  752.0
1          0         1  0.045286  0.187901  1.50 -1.07   0.000  384.0
2          0         2  0.069175  0.230659  1.41 -0.68   0.000  194.0
3          0         3  0.025769  0.149969  0.80 -0.57   0.000  104.0
4          0         4  0.023500  0.153256  0.62 -0.44   0.000   60.0
5          0         5  0.060870  0.167688  0.54 -0.30   0.000   46.0
6          0         6  0.065517  0.215416  0.70 -0.38   0.000   29.0
7          0         7  0.028696  0.218534  0.73 -0.60   0.000   23.0
8          0         8  0.186364  0.286785  0.94 -0.02   0.100   11.0
9          0         9  0.198333  0.478631  0.90 -0.94   0.175   12.0
10         0        10  0.210000  0.259422  0.50  0.00   0.130    3.0
11         0        11 -0.050000       NaN -0.05 -0.05  -0.050    1.0
12         0        12  0.605000  0.431335  0.91  0.30   0.605    2.0
13         1        

In [138]:
# 单独拿出其中一类来看， CountBid=1 和 CountAsk =1
tempTick['diff_3Tick'] = tempTick['diff_3Tick'].clip(-0.3,0.3)
tempTick['diff_1Tick'] = tempTick['diff_1Tick'].clip(-0.3,0.3)
check_index = tempTick[tempTick['CountBid'] == 1].index & tempTick[tempTick['CountAsk']==1].index 
print ("CountBid = 1 以及 CountAsk = 1 时，各个snapshot如下：")
print ("总共有 ：%s 个数据"%(len(check_index)))
print (tempTick.loc[check_index,['CountBid','CountAsk','diff_3Tick','diff_1Tick','Turnover',
                                'Past_diff2','Past_diff3','Past_diff4','Past_diff6']].head(15),'\n')
print ("CountBid = 1 以及 CountAsk = 1 时，统计的分布如下：")
print (tempTick.loc[check_index,'diff_3Tick'].value_counts(bins=10,normalize=False,sort=False))

# 可以看出，(-0.02,0.0)占比 0.29

CountBid = 1 以及 CountAsk = 1 时，各个snapshot如下：
总共有 ：271 个数据
     CountBid  CountAsk  diff_3Tick  diff_1Tick   Turnover  Past_diff2  Past_diff3  Past_diff4  Past_diff6
125       1.0       1.0        0.04       -0.01   148618.0       -0.90       -0.90       -0.10        0.73
209       1.0       1.0        0.08        0.03   222627.0        0.27        0.27        0.10        0.27
229       1.0       1.0        0.30        0.00   148658.0       -1.02        0.29        0.28       -1.22
253       1.0       1.0       -0.30       -0.12   371502.0        0.01        0.48        0.36        0.01
256       1.0       1.0        0.30        0.20   742550.0       -0.36       -0.48       -0.47        0.00
262       1.0       1.0       -0.27       -0.20   148644.0        0.00        0.00        0.00        0.47
291       1.0       1.0        0.30        0.10   148810.0        0.18        0.28        0.10        0.30
305       1.0       1.0        0.04       -0.30   745560.0        0.06        0.36    

In [131]:
###############################################################################################################################
# 收集所有数据
DataSave = pd.DataFrame()
# 数据读入 和 预处理
for date_index in range(len(TickNameList)):
    date = re.search('\d{8}',TickNameList[date_index])[0]
    tempTick = pd.read_csv(TickNameList[date_index])
    tempTran = pd.read_csv(TranNameList[date_index])
    tempTest = pd.read_csv(TestNameList[date_index])
    # Tick数据处理
    tempTick = tempTick.rename(columns={"Unnamed: 0":"Timestamp"})
    tempTick['Timestamp'] = pd.to_datetime(tempTick['Timestamp'])
    tempTick['median_5'] = (tempTick['AskP0'].rolling(window=5).median()).shift(-2).fillna(method='pad')
    tempTick['median_5'],_ = Tools_kjy.smooth(tempTick['median_5'])
    # 开平仓点提取
    tickindex = tempTest[tempTest.order != " "].index
    print ("Date : %s with %s data"%(date,len(tickindex)))
    # # Tran 数据处理
    tempTran = tempTran.rename(columns={"Unnamed: 0":"Timestamp"})
    tempTran['Timestamp'] = pd.to_datetime(tempTran['Timestamp'])
    FeatureTran = TranFeature.TranFeature(tempTran,tempTick)
    tempTick = FeatureTran._Tran_features()
    
    # past change
    def past_change(x):
        tempTick['Past_diff'+str(x)] = tempTick['Price'].diff(x).fillna(0)
    past_change(1);past_change(2);past_change(3);past_change(4);past_change(5);past_change(6)

    # future change
    tempTick['diff_3Tick'] = tempTick['Price'].diff(3).shift(-3).fillna(0)
    tempTick['diff_1Tick'] = tempTick['Price'].diff(1).shift(-1).fillna(0)
    tempTick['isadd_3Tick'] = tempTick['diff_3Tick'].apply(Tools_kjy.trinary)
    tempTick['isadd_1Tick'] = tempTick['diff_1Tick'].apply(Tools_kjy.trinary)
    # 开平仓点保存
    DataSave = DataSave.append(tempTick.loc[tickindex,:])
 
print ('done!')
DataSave = DataSave.reset_index(drop=True)
DataSave['CountBid'] = DataSave['CountBid'].clip(0,12).apply(int)
DataSave['CountAsk'] = DataSave['CountAsk'].clip(0,12).apply(int)
groups = DataSave[['CountBid','CountAsk','diff_3Tick','diff_1Tick']].groupby(['CountBid','CountAsk'])
groupsCountBA = groups.agg({'diff_1Tick':[np.mean,np.std,np.max,np.min,np.median,np.size]}).reset_index()
print (groupsCountBA.head(20))

Date : 20180502 with 119 data
_drop_Contunous costs 0.107 seconds!
Date : 20180503 with 103 data
_drop_Contunous costs 0.079 seconds!
Date : 20180504 with 107 data
_drop_Contunous costs 0.103 seconds!
Date : 20180507 with 122 data
_drop_Contunous costs 0.136 seconds!
Date : 20180508 with 91 data
_drop_Contunous costs 0.097 seconds!
Date : 20180509 with 75 data
_drop_Contunous costs 0.089 seconds!
Date : 20180510 with 54 data
_drop_Contunous costs 0.074 seconds!
Date : 20180511 with 135 data
_drop_Contunous costs 0.074 seconds!
Date : 20180514 with 123 data
_drop_Contunous costs 0.098 seconds!
Date : 20180515 with 144 data
_drop_Contunous costs 0.072 seconds!
Date : 20180516 with 86 data
_drop_Contunous costs 0.070 seconds!
Date : 20180517 with 108 data
_drop_Contunous costs 0.065 seconds!
Date : 20180518 with 96 data
_drop_Contunous costs 0.060 seconds!
Date : 20180521 with 62 data
_drop_Contunous costs 0.064 seconds!
done!
   CountBid CountAsk diff_1Tick                               

In [139]:
# 单独拿出其中一类来看  CountBid = 1 & CountAsk = 1
DataSave['diff_3Tick'] = DataSave['diff_3Tick'].clip(-0.3,0.3)
DataSave['diff_1Tick'] = DataSave['diff_1Tick'].clip(-0.3,0.3)
check_index = DataSave[DataSave['CountBid'] == 1].index & DataSave[DataSave['CountAsk']==1].index 
print ("CountBid = 1 以及 CountAsk = 1 时，各个snapshot如下：")
print ("总共有 ：%s 个数据"%(len(check_index)))
print (DataSave.loc[check_index,['CountBid','CountAsk','diff_3Tick','diff_1Tick','Turnover',
                                'Past_diff2','Past_diff3','Past_diff4','Past_diff6']].head(15),'\n')
print ("CountBid = 1 以及 CountAsk = 1 时，diff_3Tick统计的分布如下：")
print (DataSave.loc[check_index,'diff_3Tick'].value_counts(bins=10,normalize=False,sort=False))

# 可以看出，(-0.02,0.0)占比 0.29

CountBid = 1 以及 CountAsk = 1 时，各个snapshot如下：
总共有 ：65 个数据
     CountBid  CountAsk  diff_3Tick  diff_1Tick  Turnover  Past_diff2  Past_diff3  Past_diff4  Past_diff6
56          1         1       -0.02        0.28  327314.0       -0.07       -0.18       -0.26       -0.38
80          1         1        0.04        0.01  131259.0       -0.05       -0.05       -0.13       -0.12
85          1         1        0.25        0.18  131470.0        0.05        0.35        0.35        0.48
110         1         1       -0.03       -0.26  133063.0       -0.24       -0.64       -0.24        0.07
163         1         1       -0.30       -0.30  133645.0       -0.03        0.43        0.44        0.44
164         1         1        0.02       -0.03  133605.0       -0.40       -0.43       -0.43        0.04
165         1         1       -0.01        0.05  400809.0       -0.03        0.00       -0.43       -0.46
170         1         1       -0.09       -0.08  133212.0       -0.18       -0.21       -0.23  

In [154]:
# 再把位于同一区间(-0.06,0.6)的 CountAsk =1 以及 CountBid = 1拿出来
check_1 = DataSave[DataSave['diff_3Tick'] < -0.24].index & check_index
check_2 = DataSave[(DataSave['diff_3Tick'] < 0) & (DataSave['diff_3Tick'] > -0.06)].index & check_index

print (DataSave.loc[check_1,['CountBid','CountAsk','diff_3Tick','diff_1Tick','Turnover',
                                'Past_diff2','Past_diff3','Past_diff4','Past_diff6']],'\n')
print (DataSave.loc[check_2,['CountBid','CountAsk','diff_3Tick','diff_1Tick','Turnover',
                                'Past_diff2','Past_diff3','Past_diff4','Past_diff6']],'\n')

      CountBid  CountAsk  diff_3Tick  diff_1Tick   Turnover  Past_diff2  Past_diff3  Past_diff4  Past_diff6
163          1         1       -0.30       -0.30   133645.0       -0.03        0.43        0.44        0.44
327          1         1       -0.24       -0.09   264931.0       -0.15       -0.70       -0.70       -0.23
763          1         1       -0.30       -0.03   431200.0        0.00       -0.06        0.12       -0.09
764          1         1       -0.30       -0.30   143721.0        0.01       -0.36        0.00       -0.36
823          1         1       -0.30        0.02  1240231.0       -0.02        0.58        0.08        0.38
866          1         1       -0.30        0.00   145988.0        0.70        0.70        0.42       -0.07
883          1         1       -0.30       -0.09   147196.0       -0.02       -0.05       -0.27       -0.31
884          1         1       -0.30       -0.11   147178.0       -0.11       -0.11       -0.14       -0.40
1067         1         1    